# 패키지

In [1]:
import os

In [2]:
import tensorflow as tf             # 텐서플로
import tensorflow_addons as tfa     # 텐서플로 애드온
import pandas as pd                 # 판다스

/Users/chorokmoo/_MyDev/_Anaconda/anaconda3/envs/new/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Users/chorokmoo/_MyDev/_Anaconda/anaconda3/envs/new/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If 

# GPU 사용 설정

In [3]:
# 사용 가능한 모든 GPU 리스트
gpus = tf.config.list_physical_devices('GPU')

# 사용할 프로세서 선택 : GPU
if gpus:
    try: tf.config.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e: print(e)

# 데이터셋 준비

In [4]:
# 데이터셋 주소
dataset_dir = os.path.join('.', 'datasets', 'MusicalSymbol-v.1.1.5')

# CSV 파일 주소
csv_dir = os.path.join(dataset_dir, 'label.csv')

# csv 파일 불러오기
df = pd.read_csv(csv_dir)

# name을 정수값으로 추출
df['name_int'] = df['name'].str.extract('(\d+)').astype(int)

# 정수값을 기준으로 정렬 및 인덱스 초기화
df = df.sort_values('name_int')
df.reset_index(drop=True, inplace=True)

# 필요 없는 name, name_int 열 제거
df = df.drop(columns=['name', 'name_int'])

In [5]:
def column_extract(dataframe, keywords):
    # 빈 데이터 프레임 생성
    new_df = pd.DataFrame()

    # 기존 데이터 프레임 추출 시작
    for key in keywords:
        # 키워드 추출
        name_list = [s for s in dataframe.columns.to_numpy() if key in s]
        
        # 추출된 키워드 열 끼리 병합
        for n in name_list:
            # 키워드 열 끼리 or 연산
            if key in new_df:   new_df[key.strip()] = new_df[key] | dataframe[n]
            else:               new_df[key.strip()] = dataframe[n]
    
    # 새로 생성된 데이터 프레임 반환
    return new_df

def column_pick(dataframe, keywords):
    # 빈 데이터 프레임 생성
    new_df = pd.DataFrame()

    # 기존 데이터 프레임 추출 시작
    for key in keywords:
        # 키워드 추출
        name_list = [s for s in dataframe.columns.to_numpy() if key in s]

        # 추출된 키워드로 데이터 프레임 새로 만들기
        for n in name_list:
            new_df[n.strip()] = dataframe[n]
    
    # 새로 생성된 데이터 프레임 반환
    return new_df

In [6]:
# 악상 기호 전체 분류
df_all = column_extract(df, keywords=[ 
    'note', 
    'accidental', 
    'articulation', 
    'dynamic', 
    'octave', 
    'ornament', 
    'repetition', 
    'clef', 
    'key', 
    'measure', 
    'rest', 
    'time'
])

# 악상 기호 세부 분류
df_pitch        = column_pick(df, keywords=['staff-']).drop(columns=['staff-half-left', 'staff-half-right'])
df_note         = column_pick(df, keywords=['note'])
df_accidental   = column_pick(df, keywords=['accidental'])
df_dynamic      = column_pick(df, keywords=['dynamic'])
df_octave       = column_pick(df, keywords=['octave'])
df_ornament     = column_pick(df, keywords=['ornament'])
df_repetition   = column_pick(df, keywords=['repetition'])
df_clef         = column_pick(df, keywords=['clef'])
df_key          = column_pick(df, keywords=['key'])
df_measure      = column_pick(df, keywords=['measure'])
df_rest         = column_pick(df, keywords=['rest'])
df_time         = column_pick(df, keywords=['time'])

#
print(df_all.columns.to_numpy())

['note' 'accidental' 'articulation' 'dynamic' 'octave' 'ornament'
 'repetition' 'clef' 'key' 'measure' 'rest' 'time']


In [7]:
# pandas 데이터 프레임을 tensorflow 텐서로 변환
df_all          = tf.convert_to_tensor(df_all.values, dtype=tf.int16)
df_pitch        = tf.convert_to_tensor(df_pitch.values, dtype=tf.int16)
df_note         = tf.convert_to_tensor(df_note.values, dtype=tf.int16)
df_accidental   = tf.convert_to_tensor(df_accidental.values, dtype=tf.int16)
df_dynamic      = tf.convert_to_tensor(df_dynamic.values, dtype=tf.int16)
df_octave       = tf.convert_to_tensor(df_octave.values, dtype=tf.int16)
df_ornament     = tf.convert_to_tensor(df_ornament.values, dtype=tf.int16)
df_repetition   = tf.convert_to_tensor(df_repetition.values, dtype=tf.int16)
df_clef         = tf.convert_to_tensor(df_clef.values, dtype=tf.int16)
df_key          = tf.convert_to_tensor(df_key.values, dtype=tf.int16)
df_measure      = tf.convert_to_tensor(df_measure.values, dtype=tf.int16)
df_rest         = tf.convert_to_tensor(df_rest.values, dtype=tf.int16)
df_time         = tf.convert_to_tensor(df_time.values, dtype=tf.int16)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [8]:
# shape 출력
print('전체 분류')
print('--------------------------------')
print('df_all        :', df_all.shape, '\n')

print('세부 분류')
print('--------------------------------')
print('df_pitch      :', df_pitch.shape)
print('df_note       :', df_note.shape)
print('df_accidental :', df_accidental.shape)
print('df_dynamic    :', df_dynamic.shape)
print('df_octave     :', df_octave.shape)
print('df_ornament   :', df_ornament.shape)
print('df_repetition :', df_repetition.shape)
print('df_clef       :', df_clef.shape)
print('df_key        :', df_key.shape)
print('df_measure    :', df_measure.shape)
print('df_rest       :', df_rest.shape)
print('df_time       :', df_time.shape)

전체 분류
df_all        : (22617, 12) 

세부 분류
df_pitch      : (22617, 11)
df_note       : (22617, 14)
df_accidental : (22617, 5)
df_dynamic    : (22617, 18)
df_octave     : (22617, 8)
df_ornament   : (22617, 7)
df_repetition : (22617, 12)
df_clef       : (22617, 9)
df_key        : (22617, 14)
df_measure    : (22617, 6)
df_rest       : (22617, 15)
df_time       : (22617, 2)


In [9]:
@tf.function
def make_data(img_path):
    # 타겟 생성
    image = tf.io.read_file(img_path)               # 파일 로드
    image = tf.image.decode_png(image, channels=1)  # png 파일로 변환
    image = tf.cast(image, tf.float32)              # uint8 -> float32
    image = image / 255.0                           # 0~1 로 정규화
    image = 1.0 - image                             # 흑백 이미지 반전

    # 파일 이름에서 인덱스 얻기
    lable_index = tf.strings.split(img_path, os.path.sep)[-1]           # 파일 이름 추출
    lable_index = tf.strings.split(lable_index, '.')[0]                 # 확장자 제거
    lable_index = tf.strings.to_number(lable_index, out_type=tf.int32)  # 숫자 변환

    # 레이블 생성
    lable_all = tf.gather(df_all, lable_index)                          # df 에서 lable_index 번째 label 추출

    # 타겟과 레이블 반환
    return (image, lable_all)

@tf.function
def add_noise(image, label):
    # 이미지 잡음
    #noise = tf.random.normal(shape=tf.shape(image), mean=0.5, stddev=0.2, dtype=tf.float32)     # 정규 분포
    noise = tf.random.uniform(shape=tf.shape(image), minval=-0.7, maxval=0.7, dtype=tf.float32)  # 균등 분포
    image = image + noise
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

@tf.function
def rotate_image(image, label):
    # 이미지 회전
    angle = tf.random.uniform([], minval=-15, maxval=15, dtype=tf.float32)
    angle_rad = angle * (3.141592653589793 / 180.0)
    image = tfa.image.rotate(image, angle_rad)
    return image, label

def scale_image(image, label):
    # 이미지 확대 및 축소
    pass

def shift_image(image, label):
    # 이미지 상하좌우로 이동
    pass

def shake_image(image, label):
    # 이미지 흔들림
    pass

@tf.function
def filter_dataset(image, label):
    # label이 모두 0인 경우 필터링
    return tf.math.reduce_any(label != 0)

In [10]:
# 데이터셋 확인해보기
import matplotlib.pyplot as plt

def plot_images(dataset, num_images):
    plt.figure(figsize=(10, 10))
    for images, labels in dataset.take(1):
        for i in range(num_images):
            ax = plt.subplot(4, 8, i + 1)
            plt.imshow(images[i], cmap='gray')
            plt.axis('off')
    plt.show()

In [11]:
# 데이터셋 생성
dataset = tf.data.Dataset.list_files(os.path.join(dataset_dir, '*.png'))

# map : make_data
dataset = dataset.map(make_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# map : shift_image
#dataset = dataset.map(shift_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# map : rotate_image
dataset = dataset.map(rotate_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# map : scale_image
#dataset = dataset.map(scale_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# map : shake_image
#dataset = dataset.map(shake_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# map : add_noise
dataset = dataset.map(add_noise, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# filter
dataset = dataset.filter(filter_dataset)

# cache
dataset = dataset.cache()

# shuffle
dataset = dataset.shuffle(buffer_size=25000)

# repeat
#dataset = dataset.repeat(3)

# train, validation
ds_validation = dataset.take(320).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
ds_train = dataset.skip(320).batch(32)

# 만들어진 데이터셋 확인
#plot_images(ds_validation, 32)

# 모델 학습

In [12]:
# inception module
def inception_module(x, filters):
    # 1x1 conv
    out1_1 = tf.keras.layers.Conv2D(filters[0], 1, strides=1, padding='same', activation='relu')(x)
    # 1x1 conv -> 3x3 conv
    out2_1 = tf.keras.layers.Conv2D(filters[1], 1, strides=1, padding='same', activation='relu')(x)
    out2_2 = tf.keras.layers.Conv2D(filters[2], 3, strides=1, padding='same', activation='relu')(out2_1)
    # 1x1 conv -> 5x5 conv
    out3_1 = tf.keras.layers.Conv2D(filters[3], 1, strides=1, padding='same', activation='relu')(x)
    out3_2 = tf.keras.layers.Conv2D(filters[4], 5, strides=1, padding='same', activation='relu')(out3_1)
    # 3x3 max_pool -> 1x1 conv
    out4_1 = tf.keras.layers.MaxPooling2D(pool_size=3, strides=1, padding='same')(x)
    out4_2 = tf.keras.layers.Conv2D(filters[5], 1, strides=1, padding='same', activation='relu')(out4_1)

    # out1_1 + out2_2 + out3_2 + out4_4
    output = tf.keras.layers.Concatenate(axis=-1)([out1_1, out2_2, out3_2, out4_2])
    return output

In [13]:
# GoogLeNet -> MyGoogLeNet
def MyGoogLeNet(input_shape=(512, 192, 1), num_classes=12):
    # 입력
    inputs = tf.keras.layers.Input(shape=input_shape)
    # 합성곱 1
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=7, strides=2, padding='same', activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    # 합성곱 2
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1, padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D(filters=192, kernel_size=3, strides=1, padding='same', activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    # 인셉션 모듈
    x = inception_module(x, [64, 128, 96, 32, 16, 32])
    x = inception_module(x, [128, 192, 128, 96, 32, 64])
    x = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    x = inception_module(x, [192, 208, 96, 48, 16, 64])
    x = inception_module(x, [160, 224, 112, 64, 24, 64])
    x = inception_module(x, [128, 256, 128, 64, 24, 64])
    x = inception_module(x, [112, 288, 144, 64, 32, 64])
    x = inception_module(x, [256, 320, 160, 128, 32, 128])
    x = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    x = inception_module(x, [256, 320, 160, 128, 32, 128])
    x = inception_module(x, [384, 384, 192, 128, 48, 128])
    # 전역 평균 풀링
    x = tf.keras.layers.AveragePooling2D(pool_size=(16, 6))(x)
    # 출력층
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    output = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

    # 모델 생성
    model = tf.keras.Model(inputs=[inputs], outputs=[output], name='MyGoogLeNet')
    return model

In [ ]:
# 그래프 초기화
tf.keras.backend.clear_session()

# 모델 생성 및 학습
with tf.device('/device:GPU:0'):
    # 모델 생성
    model = MyGoogLeNet()

    # 모델 컴파일
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # 모델 학습
    model.fit(ds_train, epochs=4, validation_data=ds_validation)